In [1]:
!kaggle datasets download -d zynicide/wine-reviews

Traceback (most recent call last):
  File "/Users/sabrinaauger/.pyenv/versions/lewagon/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/Users/sabrinaauger/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/Users/sabrinaauger/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /Users/sabrinaauger/.kaggle. Or use the environment method.
